Tasa atractiva mínima (MARR)
===

* 60:00 min | Última modificación: Juno 22, 2019 | YouTube

**Preparación**

In [1]:
# Importa la librería financiera.
# Solo es necesario ejecutar la importación una sola vez.
import cashflows as cf

### Problema del costo de capital

* A medida que se invierte más capital los rendimientos obtenidos son menores (es más dificil acceder a inversiones con rentabilidades altas).

* A medida que se presta más capital los interses son más altos (es más dificil acceder a créditos baratos)

* Si se tiene un proyecto cuyos fondos provienen del aporte de los socios y de diferentes esquemas de financiación, ¿cómo se calculá el costo de dichos fondos?.

![assets/wacc-explain.png](assets/wacc-explain.png)

### Caso práctico

Una compañía eléctrica tiene las siguientes fuentes de financiamiento:

* Un total de `$` 4000 por la emisión de 4.000 acciones. Se espera un dividendo de `$` 0.25 por acción para los próximos años.

* Un préstamo bancario (Préstamo 1) de `$` 2.000. El préstamo se paga en 4 cuotas iguales a capital más intereses sobre el saldo total de deuda liquidados a una tasa efectiva de interés del 20%. En el momento del desembolso se cobró una comisión bancaria de `$` 50.

* Un préstamo bancario (Préstamo 2) de `$` 1.000 con descuento de 24 puntos. El préstamo se paga en 4 cuotas totales iguales que incluyen intereses más capital. La tasa de interés es del 20%.

* La venta de un bono con pago principal de `$` 5.000, el cual fue vendido por `$` 4.000. El capital se dedimirá en 4 períodos y se pagarán intereses a una tasa del 7%. El bono tiene un costo de venta de `$` 50.

El impuesto de renta es del 30%.

**Solución**

In [2]:
import cashflows as cf

In [3]:
##
## Se tienen cuatro fuentes de capital con diferentes costos
## sus datos se almacenarar en las siguientes listas:
##
monto   = [0] * 4
interes = [0] * 4

In [4]:
## emision de acciones
## --------------------------------------
monto[0]   = 4000
interes[0] = 25.0 / 1.0  # tasa de descueto de la acción

In [5]:
## préstamo 1.
## -------------------------------------------------------
## 
nrate = cf.interest_rate(const_value=[20]*5, start=2018)
credito1 = cf.fixed_ppal_loan(amount    = 2000,     # monto
                              nrate     = nrate,    # tasa de interés
                              orgpoints = 50/2000)  # costos de originación
credito1           

Beg_Ppal_Amount  Nom_Rate  Tot_Payment  Int_Payment  Ppal_Payment  \
2018              0.0      20.0         50.0          0.0           0.0   
2019           2000.0      20.0        900.0        400.0         500.0   
2020           1500.0      20.0        800.0        300.0         500.0   
2021           1000.0      20.0        700.0        200.0         500.0   
2022            500.0      20.0        600.0        100.0         500.0   

      End_Ppal_Amount  
2018           2000.0  
2019           1500.0  
2020           1000.0  
2021            500.0  
2022              0.0

En la modelación de créditos con **`cashflow`** se consideran dos tipos de costos:

* Los puntos de descuento (dispoints) como porcentaje sobre el monto de la deuda. Estos son una forma de pago de intereses por anticipado con el fin de bajar la tasa de interés del crédito. 

* Los puntos de constitución (orgpoints) como porcentaje del monto de deuda. Son los costos de constitución del crédito y no son considerados como intereses.

Ya que los intereses de los créditos pueden descontarse como costos financieros, estos disminuyen el pago del impuesto de renta. Por consiguiente, en el análisis de los créditos debe tenerse en cuenta el beneficio por pago de intereses el cual equivale a los impuestos pagados por período multiplicados por la tasa del impuesto de renta. Ya que los puntos de descuento son intereses, estos se tienen en cuenta en el cálculo de este beneficio.

In [6]:
## flujo de caja para el crédito antes de impuestos
tax_rate = cf.interest_rate(const_value=[30]*5, start=2018)
credito1.tocashflow(tax_rate=tax_rate)

2018    1950.0
2019   -1420.0
2020   -1190.0
2021    -960.0
2022    -730.0
Freq: A-DEC, dtype: float64

In [7]:
## la tasa efectiva pagada por el crédito es 
## aquella que hace el valor presente cero para
## el flujo de caja anterior (antes o después de 
## impuestos)
credito1.true_rate(tax_rate = tax_rate)

48.0427598437442

In [8]:
## se almacenan los datos para este credito
monto[1]   = 2000
interes[1] = credito1.true_rate(tax_rate = tax_rate)

In [9]:
## préstamo 2.
## -------------------------------------------------------
## 
credito2 = cf.fixed_rate_loan(amount    = 1000,     # monto
                              nrate     =   20,     # tasa de interés
                              start     = 2018,
                              grace     =    0,
                              life      =    4,     # número de cuotas
                              dispoints =    0.24)  # costos de constitución
credito2 

Beg_Ppal_Amount  Nom_Rate  Tot_Payment  Int_Payment  Ppal_Payment  \
2018      1000.000000      20.0   240.000000   240.000000      0.000000   
2019      1000.000000      20.0   386.289121   200.000000    186.289121   
2020       813.710879      20.0   386.289121   162.742176    223.546945   
2021       590.163934      20.0   386.289121   118.032787    268.256334   
2022       321.907601      20.0   386.289121    64.381520    321.907601   

      End_Ppal_Amount  
2018      1000.000000  
2019       813.710879  
2020       590.163934  
2021       321.907601  
2022         0.000000

In [10]:
credito2.tocashflow(tax_rate = tax_rate)

2018    448.000000
2019   -646.289121
2020   -597.853949
2021   -539.731744
2022   -469.985097
Freq: A-DEC, dtype: float64

In [11]:
credito2.true_rate(tax_rate = tax_rate)

132.36608595206968

In [12]:
## se almacenan los datos para este credito
monto[2]   = 1000
interes[2] = credito2.true_rate(tax_rate = tax_rate)

In [13]:
## préstamo 3.
## -------------------------------------------------------
## 
nrate = cf.interest_rate(const_value=[7]*5, start=2018)
credito3 = cf.bullet_loan(amount     = 5000,     # monto
                          nrate      =   nrate,  # tasa de interés
                          orgpoints  =    0.01,  # costos de originación
                          dispoints  =    0.20)  # puntos de descuento
credito3 

Beg_Ppal_Amount  Nom_Rate  Tot_Payment  Int_Payment  Ppal_Payment  \
2018              0.0       7.0       1050.0       1000.0           0.0   
2019           5000.0       7.0        350.0        350.0           0.0   
2020           5000.0       7.0        350.0        350.0           0.0   
2021           5000.0       7.0        350.0        350.0           0.0   
2022           5000.0       7.0       5350.0        350.0        5000.0   

      End_Ppal_Amount  
2018           5000.0  
2019           5000.0  
2020           5000.0  
2021           5000.0  
2022              0.0

In [14]:
credito3.tocashflow(tax_rate=tax_rate)

2018    2650.0
2019    -805.0
2020    -805.0
2021    -805.0
2022   -5805.0
Freq: A-DEC, dtype: float64

In [15]:
credito3.true_rate(tax_rate=tax_rate)

42.451661086842819

In [16]:
## se almacenan los datos de este crédito
monto[3]   = 5000
interes[3] = credito3.true_rate(tax_rate=tax_rate)

In [17]:
## montos
monto

[4000, 2000, 1000, 5000]

In [18]:
## tasas
interes

[25.0, 48.0427598437442, 132.36608595206968, 42.451661086842819]

In [19]:
## Costo ponderado del capital (WACC)
## -------------------------------------------------------------
## es el promdio ponderado de las tasas por 
## el porcentaje de capital correspondiente a cada fuente
##
s    = sum(monto)           # capital total
wacc = sum([x*r/s for x, r in zip(monto, interes)])
wacc

45.05915925614768

---

### Bibliografía utilizada

* [1] SAS/ETS 14.1 User's Guide, 2015.   
* [2] **hp** 12c platinum financial calculator. User's guide. 
* [3] HP Business Consultant II Owner's manual.
* [4] C.S. Park and G.P. Sharp-Bette. Advanced Engineering Economics. John Wiley & Sons, Inc., 1990.